<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/Sem%C3%A1foro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

st.title("Dashboard Semáforo de Fechas PO vs EXW Real")
st.write("Carga tu archivo Excel para visualizar el semáforo de fechas:")

archivo = st.file_uploader("Sube tu archivo Excel", type=["xlsx"])

if archivo:
    df = pd.read_excel(archivo)

    # Convertir a fecha
    df['Fecha de PO'] = pd.to_datetime(df['Fecha de PO'], errors='coerce')
    df['EXW Real'] = pd.to_datetime(df['EXW Real'], errors='coerce')

    # Solo trabajar con filas donde EXW Real NO está vacía
    df_validas = df[df['EXW Real'].notna()].copy()

    # Calcular diferencia de días solo en filas válidas
    df_validas['Diferencia días calculada'] = (df_validas['EXW Real'] - df_validas['Fecha de PO']).dt.days

    # Semáforo solo para filas válidas
    def status_semáforo(dias):
        if pd.isna(dias):
            return 'Sin fecha'
        elif 0 <= dias <= 6:
            return 'En tiempo'
        elif dias >= 7:
            return 'On Delay'
        else:
            return 'Revisar'

    df_validas['Semáforo'] = df_validas['Diferencia días calculada'].apply(status_semáforo)

    # Mostrar tabla solo con filas válidas
    st.subheader("Tabla con Semáforo (solo registros con EXW Real)")
    st.dataframe(df_validas[['Fecha de PO', 'EXW Real', 'Diferencia días calculada', 'Semáforo']])

    # Gauge chart con Plotly SOLO con datos válidos
    st.subheader("Indicador Gauge: % En Tiempo")

    total_validos = df_validas['Semáforo'].isin(['En tiempo', 'On Delay']).sum()
    en_tiempo = (df_validas['Semáforo'] == 'En tiempo').sum()

    if total_validos > 0:
        porcentaje_en_tiempo = en_tiempo / total_validos * 100
    else:
        porcentaje_en_tiempo = 0

    fig = go.Figure(go.Indicator(
        mode = "gauge+number+delta",
        value = porcentaje_en_tiempo,
        title = {'text': "% En Tiempo (0-6 días)"},
        gauge = {
            'axis': {'range': [None, 100]},
            'steps': [
                {'range': [0, 60], 'color': "#F5B7B1"},    # rojo
                {'range': [60, 85], 'color': "#F9E79F"},   # amarillo
                {'range': [85, 100], 'color': "#ABEBC6"}   # verde
            ],
            'threshold': {
                'line': {'color': "black", 'width': 4},
                'thickness': 0.75,
                'value': porcentaje_en_tiempo
            }
        }
    ))

    st.plotly_chart(fig)

    st.info("Dashboard generado por Isra 🤖")

else:
    st.warning("Sube un archivo para ver el dashboard.")

